# Goal

Our goal is to visualize the types of hospitalizations by major diagnostic category (MDC) using a lab value such as glucose:

https://idl.uw.edu/mosaic-framework-example/observable-latency

In [1]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [22]:
%%capture
%%sql
WITH glucose_measurements AS (
    SELECT 
        l.subject_id,
        l.hadm_id,
        l.valuenum AS glucose_value,
        l.charttime AS measurement_time
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/labevents.parquet') l
    JOIN 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/d_labitems.parquet') d
    ON 
        l.itemid = d.itemid
    WHERE 
        d.label = 'Glucose'
),
drgcodes AS (
    SELECT 
        subject_id,
        hadm_id,
        drg_type,
        drg_code
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/drgcodes.parquet')
    WHERE 
        drg_type = 'HCFA'
)
SELECT 
    g.subject_id,
    g.hadm_id,
    g.glucose_value,
    g.measurement_time,
    d.drg_type,
    d.drg_code
FROM 
    glucose_measurements g
JOIN 
    drgcodes d
ON 
    g.subject_id = d.subject_id
    AND g.hadm_id = d.hadm_id
ORDER BY 
    g.subject_id, g.hadm_id, g.measurement_time

In [23]:
%%capture
%%sql
WITH glucose_measurements AS (
    SELECT 
        l.subject_id,
        l.hadm_id,
        l.valuenum AS glucose_value
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/labevents.parquet') l
    JOIN 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/d_labitems.parquet') d
    ON 
        l.itemid = d.itemid
    WHERE 
        d.label = 'Glucose'
        AND l.valuenum IS NOT NULL
),
drgcodes AS (
    SELECT 
        subject_id,
        hadm_id,
        drg_type,
        drg_code
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/drgcodes.parquet')
    WHERE 
        drg_type = 'HCFA'
),
ms_drg_to_mdc AS (
    SELECT
        ms_drg,
        mdc
    FROM
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/ms_drg_to_mdc.parquet')
),
mdc_dictionary AS (
    SELECT
        mdc_number,
        mdc_description
    FROM
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/mdc_dictionary.parquet')
),
mdc_counts AS (
    SELECT
        m.mdc,
        COUNT(*) AS mdc_count
    FROM
        glucose_measurements g
    JOIN 
        drgcodes d ON g.subject_id = d.subject_id AND g.hadm_id = d.hadm_id
    LEFT JOIN
        ms_drg_to_mdc m ON CAST(d.drg_code AS INTEGER) = m.ms_drg
    GROUP BY
        m.mdc
    HAVING
        COUNT(*) >= 20
)
SELECT 
    g.subject_id,
    g.hadm_id,
    g.glucose_value,
    d.drg_type,
    d.drg_code,
    m.mdc,
    md.mdc_description
FROM 
    glucose_measurements g
JOIN 
    drgcodes d ON g.subject_id = d.subject_id AND g.hadm_id = d.hadm_id
LEFT JOIN
    ms_drg_to_mdc m ON CAST(d.drg_code AS INTEGER) = m.ms_drg
LEFT JOIN
    mdc_dictionary md ON m.mdc = md.mdc_number
INNER JOIN
    mdc_counts mc ON m.mdc = mc.mdc
ORDER BY 
    g.subject_id, g.hadm_id, g.glucose_value

In [13]:
df = _

In [15]:
df['mdc_description'].value_counts()

mdc_description
Diseases and disorders of the circulatory system                                   577230
Diseases and disorders of the digestive system                                     259581
Infectious and parasitic diseases, systemic or unspecified sites                   241405
Diseases and disorders of the nervous system                                       211832
Diseases and disorders of the respiratory system                                   197244
Diseases and disorders of the hepatobiliary system and pancreas                    170848
Diseases and disorders of the musculoskeletal system and connective tissue         154653
Myeloproliferative diseases and disorders, poorly differentiated neoplasms         122313
Diseases and disorders of the kidney and urinary tract                             120123
Endocrine, nutritional and metabolic diseases and disorders                         76757
Injuries, poisonings and toxic effects of drugs                                     

# Aligning charttimes to the last measurement for every hospitalization

In [24]:
%%capture
%%sql 
WITH glucose_measurements AS (
    SELECT 
        l.subject_id,
        l.hadm_id,
        l.valuenum AS glucose_value,
        l.charttime AS measurement_time
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/labevents.parquet') l
    JOIN 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/d_labitems.parquet') d
    ON 
        l.itemid = d.itemid
    WHERE 
        d.label = 'Glucose'
),
drgcodes AS (
    SELECT 
        subject_id,
        hadm_id,
        drg_type,
        drg_code
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/drgcodes.parquet')
    WHERE 
        drg_type = 'HCFA'
),
last_measurement_times AS (
    SELECT
        subject_id,
        hadm_id,
        MAX(measurement_time) AS last_measurement_time
    FROM
        glucose_measurements
    GROUP BY
        subject_id, hadm_id
)
SELECT 
    g.subject_id,
    g.hadm_id,
    g.glucose_value,
    g.measurement_time,
    TIMESTAMP '2000-01-01 00:00:00' + 
        INTERVAL (DATEDIFF('second', g.measurement_time, lmt.last_measurement_time)) SECOND 
        AS relative_measurement_time,
    d.drg_type,
    d.drg_code
FROM 
    glucose_measurements g
JOIN 
    drgcodes d ON g.subject_id = d.subject_id AND g.hadm_id = d.hadm_id
JOIN
    last_measurement_times lmt ON g.subject_id = lmt.subject_id AND g.hadm_id = lmt.hadm_id
ORDER BY 
    g.subject_id, g.hadm_id, g.measurement_time

In [26]:
%%capture
%%sql 

WITH glucose_measurements AS (
    SELECT 
        l.subject_id,
        l.hadm_id,
        l.valuenum AS glucose_value,
        l.charttime AS measurement_time
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/labevents.parquet') l
    JOIN 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/d_labitems.parquet') d
    ON 
        l.itemid = d.itemid
    WHERE 
        d.label = 'Glucose'
        AND l.valuenum IS NOT NULL
),
drgcodes AS (
    SELECT 
        subject_id,
        hadm_id,
        drg_type,
        drg_code
    FROM 
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/drgcodes.parquet')
    WHERE 
        drg_type = 'HCFA'
),
ms_drg_to_mdc AS (
    SELECT
        ms_drg,
        mdc
    FROM
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/ms_drg_to_mdc.parquet')
),
mdc_dictionary AS (
    SELECT
        mdc_number,
        mdc_description
    FROM
        read_parquet('~/data/physionet.org/processed/mimiciv/hosp/mdc_dictionary.parquet')
),
mdc_counts AS (
    SELECT
        m.mdc,
        COUNT(*) AS mdc_count
    FROM
        glucose_measurements g
    JOIN 
        drgcodes d ON g.subject_id = d.subject_id AND g.hadm_id = d.hadm_id
    LEFT JOIN
        ms_drg_to_mdc m ON CAST(d.drg_code AS INTEGER) = m.ms_drg
    GROUP BY
        m.mdc
    HAVING
        COUNT(*) >= 20
),
last_measurement_times AS (
    SELECT
        subject_id,
        hadm_id,
        MAX(measurement_time) AS last_measurement_time
    FROM
        glucose_measurements
    GROUP BY
        subject_id, hadm_id
)
SELECT 
    g.subject_id,
    g.hadm_id,
    g.glucose_value,
    g.measurement_time,
    TIMESTAMP '2000-01-01 00:00:00' + 
        INTERVAL (DATEDIFF('second', g.measurement_time, lmt.last_measurement_time)) SECOND 
        AS relative_measurement_time,
    d.drg_type,
    d.drg_code,
    m.mdc,
    md.mdc_description
FROM 
    glucose_measurements g
JOIN 
    drgcodes d ON g.subject_id = d.subject_id AND g.hadm_id = d.hadm_id
LEFT JOIN
    ms_drg_to_mdc m ON CAST(d.drg_code AS INTEGER) = m.ms_drg
LEFT JOIN
    mdc_dictionary md ON m.mdc = md.mdc_number
INNER JOIN
    mdc_counts mc ON m.mdc = mc.mdc
JOIN
    last_measurement_times lmt ON g.subject_id = lmt.subject_id AND g.hadm_id = lmt.hadm_id
ORDER BY 
    g.subject_id, g.hadm_id, g.measurement_time